In [17]:
try :
    # % tensorflow_version only exists in Colab .
    % tensorflow_version 2. x
except Exception :
    pass

import tensorflow as tf
import numpy as np
import pandas as pd
import stockstats
import os

from lib.rens import rens

In [16]:
def create_dataset(dataset, look_back = 1):
  
    dataX = []
    for i in range(len(dataset)-look_back):
        dataX.append(dataset[i:(i+look_back)])

        #dataX = np.array([dataset.iloc[i] for i in range(len(dataset)-look_back)])
        dataY = np.array([dataset[i+look_back] for i in range(len(dataset)-look_back)])

        # dataX = dataX/np.amax(dataX)
        # dataY = dataY/np.amax(dataY)

        # (nb_sequences, size_sequence, nb_channels)
        dataX = np.reshape(dataX, (dataset.shape[0]-look_back, look_back, 1))
        return np.array(dataX), np.array(dataY)

In [11]:
def make_ready(filer):
    for i, fil in enumerate(filer):
        print(f"{i+1} av {len(filer)} renset" ,end='\r')
        data = cleanup.clean_data(pd.read_excel(fil))
        stock = stockstats.StockDataFrame.retype(data)
        indikatorer = ['rsi_20', 'trix', 'open_8_sma', 'open_16_sma', 'macds', 'open_30_sma']
        for ind in indikatorer:
            stock.get(ind)
        stock['sma8-16'] = [stock.open_8_sma[i] - stock.open_16_sma[i] for i, value in enumerate(stock.open_8_sma)]
        stock['sma30_derivert'] = [stock.open_30_sma[i] / stock.open_30_sma[i-1] if i>1 else 1 
                                   for i, value in enumerate(stock.open_30_sma)]

        #evaluate_trades(stock)
        break # må tas bort ettervært
    
    return stock

    
def add_data(word):
    ordet = 'data/' + word
    return ordet

In [13]:
cleanup = rens()

In [15]:
filer = list(map(add_data, os.listdir('data/')))[4:5] # 60 første er lagd til å bygge modell
data = make_ready(filer)
data

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


,tid,open,close,high,low,volume,amount,close_-1_s,close_-1_d,rs_20,rsi_20,trix,open_8_sma,open_16_sma,macd,macds,macdh,open_30_sma,sma8-16,sma30_derivert
0,9:0,154.30,154.30,154.30,154.30,19041,19041,NaN,NaN,NaN,NaN,NaN,154.300000,154.300000,0.000000,0.000000,0.000000,154.300000,0.000000,1.000000
1,9:1,153.40,153.40,153.40,153.40,9073,9073,154.30,-0.90,0.000000,0.000000,-0.092699,153.850000,153.850000,-0.020192,-0.011218,-0.008974,153.850000,0.000000,1.000000
2,9:2,154.50,154.50,154.50,154.50,11473,11473,153.40,1.10,1.286550,56.265985,-0.026206,154.066667,154.066667,0.008382,-0.003185,0.011567,154.066667,0.000000,1.001408
3,9:3,152.85,152.85,152.85,152.85,11400,11400,154.50,-1.65,0.424409,29.795424,-0.036276,153.762500,153.762500,-0.038400,-0.015114,-0.023286,153.762500,0.000000,0.998026
4,9:4,153.50,153.50,153.50,153.50,1324,1324,152.85,0.65,0.702289,41.255560,-0.035721,153.710000,153.710000,-0.037147,-0.021669,-0.015479,153.710000,0.000000,0.999659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,16:16,149.50,149.50,149.50,149.50,6470,6470,149.25,0.25,0.830436,45.368205,-0.020484,149.256250,149.450000,-0.187746,-0.180478,-0.007268,149.686667,-0.193750,1.000056
406,16:17,149.50,149.50,149.50,149.50,6808,6808,149.50,0.00,0.830436,45.368205,-0.019663,149.262500,149.428125,-0.163871,-0.177157,0.013286,149.696667,-0.165625,1.000067
407,16:18,149.65,149.65,149.65,149.65,961,961,149.50,0.15,0.955063,48.850752,-0.017988,149.312500,149.415625,-0.131332,-0.167992,0.036660,149.696667,-0.103125,1.000000
408,16:19,150.00,150.00,150.00,150.00,4208,4208,149.65,0.35,1.261164,55.774998,-0.014985,149.412500,149.425000,-0.076421,-0.149678,0.073256,149.701667,-0.012500,1.000033


In [19]:
tf.__version__

'1.14.0'